### Firstly setting up environment variables



Add your path there instead and copy paste in terminal in your nnUNet Environment:
```
export nnUNet_raw_data_base="/location/to/nnUNet_raw_data_base"
export nnUNet_preprocessed="/location/to/nnUNet_preprocessed"
export RESULTS_FOLDER="/location/to/nnUNet_trained_models"
```

### Step 1: Transferring of Data

In [3]:
import os 
from natsort import natsorted
from glob2 import glob

In [4]:
def nnUnet_transfer_images(patients_dir, list_of_modalities, output_dir, exp_name):
    all_patients_dir = natsorted(glob(patients_dir + '/*'))
    
    for i, p in enumerate(all_patients_dir):
        for j, modality in enumerate(list_of_modalities):
            src_file = p + '/' + p.split('/')[-1] + '_' + modality + '.nii.gz'
            dst_file = output_dir + '/' + exp_name + '_' + '{:04}'.format(i+1) + '_000' + str(j) + '.nii.gz' 

            #print(dst_file)
            os.system('cp -r '+ src_file + ' ' + dst_file)    
            
def nnUnet_transfer_segs(patients_dir, output_dir, exp_name):
    
    all_patients_dir = natsorted(glob(patients_dir + '/*'))
    
    for i, p in enumerate(all_patients_dir):
        src_file = p + '/' + p.split('/')[-1] + '_seg.nii.gz'
        dst_file = output_dir + '/' + exp_name + '_'  + '{:04}'.format(i+1) + '.nii.gz'

        #print(dst_file)
        os.system('cp -r '+ src_file + ' ' + dst_file)  

#### Creating Files directory

Change the experiment name and path for your project/experiment

In [13]:
os.system('mkdir /location/to/TaskXXX_UCSFPostopADataReleaseFinal')
os.system('mkdir /location/to/TaskXXX_UCSFPostopADataReleaseFinal/imagesTr')
os.system('mkdir /location/to/nnUNet_raw_data/TaskXXX_UCSFPostopADataReleaseFinal/labelsTr')
os.system('mkdir /location/to/TaskXXX_UCSFPostopADataReleaseFinal/imagesTs')

0

#### Transferring Images

Change the experiment name and path for your project/experiment

In [14]:
nnUnet_transfer_images(patients_dir = '/location/to/UCSF_POSTOP_GLIOMA_DATASET_FINAL_TRAIN/', 
                     list_of_modalities = ['flair','t1','t2','t1ce'], 
                     output_dir = '/location/to/TaskXXX_UCSFPostopADataReleaseFinal/imagesTr', 
                     exp_name = 'TaskXXX_UCSFPostopADataReleaseFinal')

#### Transferring Segs

Change the experiment name and path for your project/experiment

In [15]:
nnUnet_transfer_segs(patients_dir = '/location/to/UCSF_POSTOP_GLIOMA_DATASET_FINAL_TRAIN/', 
                     output_dir = '/location/to/TaskXXX_UCSFPostopADataReleaseFinal/labelsTr', 
                     exp_name = 'TaskXXX_UCSFPostopADataReleaseFinal')

#### Creating dataset json

In [16]:
from typing import Tuple
import numpy as np
from batchgenerators.utilities.file_and_folder_operations import *

In [17]:
def get_identifiers_from_splitted_files(folder: str):
    uniques = np.unique([i[:-12] for i in subfiles(folder, suffix='.nii.gz', join=False)])
    return uniques


def generate_dataset_json(output_file: str, imagesTr_dir: str, imagesTs_dir: str, modalities: Tuple,
                          labels: dict, dataset_name: str, license: str = "hands off!", dataset_description: str = "",
                          dataset_reference="", dataset_release='0.0'):
    """
    :param output_file: This needs to be the full path to the dataset.json you intend to write, so
    output_file='DATASET_PATH/dataset.json' where the folder DATASET_PATH points to is the one with the
    imagesTr and labelsTr subfolders
    :param imagesTr_dir: path to the imagesTr folder of that dataset
    :param imagesTs_dir: path to the imagesTs folder of that dataset. Can be None
    :param modalities: tuple of strings with modality names. must be in the same order as the images (first entry
    corresponds to _0000.nii.gz, etc). Example: ('T1', 'T2', 'FLAIR').
    :param labels: dict with int->str (key->value) mapping the label IDs to label names. Note that 0 is always
    supposed to be background! Example: {0: 'background', 1: 'edema', 2: 'enhancing tumor'}
    :param dataset_name: The name of the dataset. Can be anything you want
    :param license:
    :param dataset_description:
    :param dataset_reference: website of the dataset, if available
    :param dataset_release:
    :return:
    """
    train_identifiers = get_identifiers_from_splitted_files(imagesTr_dir)

    if imagesTs_dir is not None:
        test_identifiers = get_identifiers_from_splitted_files(imagesTs_dir)
    else:
        test_identifiers = []

    json_dict = {}
    json_dict['name'] = dataset_name
    json_dict['description'] = dataset_description
    json_dict['tensorImageSize'] = "4D"
    json_dict['reference'] = dataset_reference
    json_dict['licence'] = license
    json_dict['release'] = dataset_release
    json_dict['modality'] = {str(i): modalities[i] for i in range(len(modalities))}
    json_dict['labels'] = {str(i): labels[i] for i in labels.keys()}

    json_dict['numTraining'] = len(train_identifiers)
    json_dict['numTest'] = len(test_identifiers)
    json_dict['training'] = [
        {'image': "./imagesTr/%s.nii.gz" % i, "label": "./labelsTr/%s.nii.gz" % i} for i
        in
        train_identifiers]
    json_dict['test'] = ["./imagesTs/%s.nii.gz" % i for i in test_identifiers]

    if not output_file.endswith("dataset.json"):
        print("WARNING: output file name is not dataset.json! This may be intentional or not. You decide. "
              "Proceeding anyways...")
    save_json(json_dict, os.path.join(output_file))

In [18]:
generate_dataset_json(
    output_file = "/location/to/TaskXXX_UCSFPostopADataReleaseFinal/dataset.json",
    imagesTr_dir = "/location/to/TaskXXX_UCSFPostopADataReleaseFinal/imagesTr",
    imagesTs_dir = "/location/to/TaskXXX_UCSFPostopADataReleaseFinal/imagesTs",
    modalities = ('flair', 't1', 't2', 't1ce'),
    labels = {
        "0": "background",
        "1": "Tumor_Necrosis",
        "2": "Edema",
        "3": "Enhancing Tumor",
        "4": "Resection Cavity"
    },
    dataset_name = "UCSF-LPTDG",
    license = "hands off",
    dataset_description = "nnUnet experiment for UCSF-LPTDG",
    dataset_reference = "",
    dataset_release = "1.0"
)

### Step 2: Run Preprocessing


Copy and run this on the terminal.

``` 
nnUNet_plan_and_preprocess -t XXX --verify_dataset_integrity
```

If everything is okay, then it will automatically run preprocessing. If not, there could be errors in your image and seg size then you need to fix it. OR if there is some small orientation differences also, it won't run automatically, here you can still do preprocessing by running the following

```
nnUNet_plan_and_preprocess -t XXX
```

### Step 3: Run training

To run the training run the following on the terminal

```
CUDA_VISIBLE_DEVICES=0 nnUNet_train 3d_fullres nnUNetTrainerV2 TaskXXX_UCSFPostopADataReleaseFinal all --npz
```

FOLD = 0,1,2,3,4,all

To stop training anytime, just press CTRL-C

To start again
```
CUDA_VISIBLE_DEVICES=0 nnUNet_train 3d_fullres nnUNetTrainerV2 TaskXXX_UCSFPostopADataReleaseFinal all --npz -c
```

### Step 4: Preprocess Test Set



In [19]:
import os 
from natsort import natsorted
from glob2 import glob

#make folders for reorganizes 
os.system('mkdir /location/to/Task180_UCSFPostopADataReleaseFinal_Test_raw')          
#prediction output dir
os.system('mkdir /location/to/Task180_UCSFPostopADataReleaseFinal_Test_pred')
#prediction output dir for orignal naming
os.system('mkdir /location/to/Task180_UCSFPostopADataReleaseFinal_Test_pred_original')


#transfer test set images
nnUnet_transfer_images(patients_dir = '/data/Data/UCSF_POSTOP_GLIOMA_DATASET_FINAL_TEST/', 
                     list_of_modalities = ['flair','t1','t2','t1ce'], 
                     output_dir = '/location/to/TaskXXX_UCSFPostopADataReleaseFinal_Test_raw', 
                     exp_name = 'TaskXXX_UCSFPostopADataReleaseFinal_Test')


### Step 4: Inference


To run the inference run the following on the terminal

```
CUDA_VISIBLE_DEVICES=0 nnUNet_predict -i /location/to/Task180_UCSFPostopADataReleaseFinal_Test_raw -o /home/jrudie/Desktop/NNUnet/Predictions/Task180_UCSFPostopADataReleaseFinal_Test_pred -t 180 -m 3d_fullres -f all 

```



In [21]:
### Convert segmentation names back to original

from glob2 import glob
from natsort import natsorted
import os

### Definining and sorted the patients directory and predictions direcroty

validation_image_list = natsorted(
    glob('/location/to/UCSF_POSTOP_GLIOMA_DATASET_FINAL_TEST/*'))

pred_segs_list = natsorted(
    glob('/location/to/Task180_UCSFPostopADataReleaseFinal_Test_pred/*nii.gz'))


### Getting all patients in the same order as pred_segs

validation_patients = [i.split('/')[-1] for i in validation_image_list]

### Copying in the same order

for i, pat in enumerate(validation_patients):
    src_file = pred_segs_list[i]
    dst_file = '/location/to/Task180_UCSFPostopADataReleaseFinal_Test_pred_original/' + pat + '.nii.gz'
    os.system('cp -r ' + src_file + ' ' + dst_file)
    
### Done